# TASK 1: Recommender System Challenge

---

**Student Name:** Abhi Gambhir

**Student Id:** 31072100

**Kaggle Name:** Abhi Gambhir

In [ ]:
# This task was performed on Kaggle notebook
!pwd

/kaggle/working


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## 1.1 Alternative Least Square (ALS) model

### 1.1.1 Reading Datasets

In [ ]:
# Reading Datasets in Kaggle notebook: the reason for path given

train_df = pd.read_csv('../input/fit5212-s12022a2/flickr_train_data.csv')
print(train_df.shape)

validation_df = pd.read_csv('../input/fit5212-s12022a2/flickr_validation_data.csv')
print(validation_df.shape)

test_df = pd.read_csv('../input/fit5212-s12022a2/flickr_test_data.csv')
print(test_df.shape)

display(train_df.sample(5))
display(validation_df.sample(5))
display(test_df.sample(5))

(110129, 3)
(346600, 3)
(346600, 2)


,user_id,item_id,rating
19517,604,7903,1
50419,1583,1142,1
43843,1362,8208,1
93826,2909,5939,1
5300,158,4090,1


,user_id,item_id,rating
151635,1516,8014,0
315308,3153,7803,0
290872,2908,7588,0
26576,265,8027,0
14794,147,1001,0


,user_id,item_id
42461,424,3032
205204,2052,8489
108103,1081,8651
19603,196,8739
5966,59,2811


In [ ]:
# reading item_fea and user_fea, though not needed for this task

item_df = pd.read_csv('../input/fit5212-s12022a2/flickr_item_fea.csv')
print(item_df.shape)
display(item_df.sample(5)) # pixels

user_df = pd.read_csv('../input/fit5212-s12022a2/flickr_user_fea.csv')
print(user_df.shape)
display(user_df.sample(5)) # pixels


(9004, 257)


,Unnamed: 0,fea_0,fea_1,fea_2,fea_3,fea_4,fea_5,fea_6,fea_7,fea_8,...,fea_246,fea_247,fea_248,fea_249,fea_250,fea_251,fea_252,fea_253,fea_254,fea_255
5521,5521,-2.306857,-1.948233,-2.664597,-3.002338,-1.938732,-1.916334,0.235660,-2.898138,-1.693517,...,-1.899717,-0.660084,-3.906935,-1.973635,-3.029933,-1.582268,-1.737332,-2.901316,-2.310423,-1.605248
8606,8606,-3.367962,-2.047345,-1.727935,-3.002760,-3.973128,-1.892427,-2.938514,-3.155282,-2.062454,...,-4.699474,-2.165234,-2.665777,-2.569584,-3.224922,-4.306575,-4.130551,-1.816162,-4.084342,-2.135868
8466,8466,-0.580237,-0.484222,-0.406011,-0.678818,-0.389216,0.275304,-0.672962,-0.419860,-0.260209,...,-0.577435,-0.324893,-0.273719,-1.231585,-0.521595,0.187549,-0.380257,-0.326531,-0.655827,-0.620268
6352,6352,-1.110793,-1.303758,-1.422959,-1.923243,-1.580022,-0.496555,-1.673259,-0.578904,-1.477738,...,-0.677649,-1.207844,-0.688512,-0.535776,-1.200206,-1.250671,-0.674937,-1.515947,-1.659804,-0.644525
6856,6856,-0.843265,-1.102187,-2.465615,-2.243539,-2.932255,-3.410351,-1.604622,-2.196860,-2.500995,...,-3.337695,-4.154559,-1.943396,-2.506057,-3.420659,-1.894251,-3.251228,-2.185389,-2.995737,-2.419604


(3466, 257)


,Unnamed: 0,fea_0,fea_1,fea_2,fea_3,fea_4,fea_5,fea_6,fea_7,fea_8,...,fea_246,fea_247,fea_248,fea_249,fea_250,fea_251,fea_252,fea_253,fea_254,fea_255
1297,1297,-1.364617,-1.176425,-1.027739,-1.824824,-1.445845,-1.145369,-1.267288,-1.260870,-1.000198,...,-1.527213,-1.001170,-1.336085,-1.478707,-1.635804,-1.222542,-1.073873,-1.092688,-1.700482,-1.193177
139,139,-1.466434,-1.251319,-1.217897,-1.224017,-1.557358,-1.338939,-1.495694,-1.495909,-1.353752,...,-1.546439,-1.261565,-1.619112,-1.290496,-1.543117,-0.842788,-1.003908,-1.154666,-1.669511,-0.994128
683,683,-1.931086,-1.896037,-2.536857,-1.923917,-1.843758,-1.956919,-1.930568,-2.194861,-2.098782,...,-2.352095,-1.956845,-1.802881,-1.641021,-2.147817,-2.592563,-2.313086,-1.879816,-2.320742,-2.050350
351,351,-1.338499,-0.989377,-0.990149,-1.617422,-1.277687,-0.884367,-1.155983,-1.103804,-0.964187,...,-1.209409,-0.888793,-1.231778,-1.066410,-1.323845,-1.025251,-0.951644,-1.000838,-1.520991,-0.745077
380,380,-1.160855,-1.160009,-0.924939,-1.748364,-1.422211,-1.107048,-1.455246,-1.422243,-1.013346,...,-1.490657,-0.943698,-1.337642,-1.325325,-1.620294,-1.097551,-0.786549,-1.056659,-1.779142,-0.950352


In [ ]:
# installing implicit package

!pip install implicit

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import implicit
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
import scipy.sparse as sparse

In [ ]:
# TODO CHANGE CC
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


In [ ]:
display(train_df.head())
display(validation_df.head())
display(test_df.head())

,user_id,item_id,rating
0,0,0,1
1,0,1,1
2,0,2,1
3,0,3,1
4,0,4,1


,user_id,item_id,rating
0,0,20,1
1,0,3260,0
2,0,390,0
3,0,5425,0
4,0,8631,0


,user_id,item_id
0,0,8929
1,0,8906
2,0,8838
3,0,8821
4,0,8756


In [ ]:
# checking shape of datasets

print(train_df.shape)
print(validation_df.shape)
print(test_df.shape)

(110129, 3)
(346600, 3)
(346600, 2)


In [ ]:
# changing datatype to 'categorical'
train_df['user_id'] = train_df['user_id'].astype('category')
train_df['item_id'] = train_df['item_id'].astype('category')

# cat.codes: Returns series of codes as well as the index
train_df['user_id'] = train_df['user_id'].cat.codes
train_df['item_id'] = train_df['item_id'].cat.codes

In [ ]:
# checking the min max of item_id and user_id
train_df.describe()

,user_id,item_id,rating
count,110129.000000,110129.000000,110129.0
mean,1711.431113,4108.175158,1.0
std,989.248533,2569.085090,0.0
min,0.000000,0.000000,1.0
25%,856.000000,1860.000000,1.0
50%,1707.000000,3962.000000,1.0
75%,2586.000000,6252.000000,1.0
max,3465.000000,9003.000000,1.0


In [ ]:
# convert to sparse matrix
sparse_item_user = sparse.csr_matrix((train_df['rating'].astype(float), (train_df['item_id'], train_df['user_id'])))
sparse_user_item = sparse.csr_matrix((train_df['rating'].astype(float), (train_df['user_id'], train_df['item_id'])))


> Sparse matrix is used to introduce the 0's and make our computations faster. Even by converting user_id and item_id to categorical datatype, it reduces computation time.

In [ ]:
# sparse matrix where rows are items and columns are users
sparse_item_user


<9004x3466 sparse matrix of type '<class 'numpy.float64'>'
	with 110129 stored elements in Compressed Sparse Row format>

In [ ]:
# sparse matrix where rows are users and columns are items
sparse_user_item


<3466x9004 sparse matrix of type '<class 'numpy.float64'>'
	with 110129 stored elements in Compressed Sparse Row format>

In [ ]:
sparse_item_user[1,:].todense()


matrix([[1., 0., 0., ..., 0., 0., 0.]])

### 1.1.2 Evaluate Recommendation Engine using NDCG

Reference: https://towardsdatascience.com/evaluate-your-recommendation-engine-using-ndcg-759a851452d1

**NDCG :** Normalized Discounted Cumulative Gain

Since a recommendation recommends the user from a super-set of documents (images in our case), recommendation can be considered simply performing document retrieval task. In document retrieval tasks, NDCG measure can be used.

We will be using DCG (Discounted Cumulative Gain) with NDCG, as DCG considers the position along with relevancec i.e order of recommendation. The computation involves dividing the relevance score by log of corresponding position.


$$ D C G=\sum_{i=1}^{n} \frac{\text { relevance }_{i}}{\log _{2}(i+1)} $$

To evaluate a recommendation engine, compute a mean of NDCG.

We can evaluate our recommendation performance on Validation dataset given, which we will be doing in the coming steps.

In [ ]:
import math

def NDCG(true_validation, pred_validation):
    
    # to store DCG values
    DCG = []
    
    # we will be considering only the rows which have rating 1, and if noticed validation
    # dataset has one such row for every user
    
    positive_rated = true_validation[true_validation['rating'] == 1]
    
    for i in range(len(positive_rated)):
        
        # storing user_id and corresponding item_id
        # as mentioned each user has only one positive_rated ('1') item.
        
        user_id = positive_rated.iloc[i]['user_id']
        item_id = positive_rated.iloc[i]['item_id']
        
        # items we recommend
        items_recommended = list(pred_validation[pred_validation['user_id'] == i]['item_id'])    
        
        # if the validation true rated value item is present in our 15 recommended items, calculate DCG
        # other wise take 0
        
        if item_id in items_recommended:
            
            rank = items_recommended.index(item_id) + 1   # adding 1 as index starts with 0
            
            if rank == 1:
                DCG.append(1)
            else:
                DCG.append(1/math.log(rank, 2))
                
        else: 
            DCG.append(0)
            
    return np.mean(DCG)

In [ ]:
# Reference: Recommender_Systems_Part_2 Tutorial Code

def recommend(user_id, sparse_user_item, user_vecs, item_vecs, test_df):
    
    # Get the interactions scores from the sparse person content matrix
    user_interactions = sparse_user_item[user_id,:].toarray()
    
    # Add 1 to everything, so that articles with no interaction yet become equal to 1
    user_interactions = user_interactions.reshape(-1) + 1
    
    # Make articles already interacted zero
    user_interactions[user_interactions > 1] = 0
    
    # Get dot product of person vector and all content vectors
    rec_vector = user_vecs[user_id,:].dot(item_vecs.T).toarray()
    
    # Scale this recommendation vector between 0 and 1
    min_max = MinMaxScaler()
    rec_vector_scaled = min_max.fit_transform(rec_vector.reshape(-1,1))[:,0]
    
    # Content already interacted have their recommendation multiplied by zero
    recommend_vector = user_interactions * rec_vector_scaled

    # Start empty list to store items and scores
    items = []
    scores = []
    
    test_item_id = test_df[test_df['user_id'] == user_id]['item_id']
    
    for i in test_item_id:
        items.append(i)
        scores.append(recommend_vector[i])
        
    # record the score
    recommendations = pd.DataFrame({'user_id': user_id, 'item_id': items, 'score': scores})
    
    # top 15 items by sorting descending the scores
    recommendations = recommendations.sort_values(by='score', ascending = False)[:15]
    
    return recommendations

In [ ]:
# defining validation ndcg scoring function

def validation_ndcg_scoring(user_vecs, item_vecs, validation_df):
    
    user_id_list = []
    item_id_list = []
    
    # for each unique users in validation dataset
    for user_id in set(validation_df['user_id']):
        
        # recommend 15 items by calling recommend()
        recommendations = recommend(user_id, sparse_user_item, user_vecs, item_vecs, validation_df)
        
        # append lists to empty list
        user_id_list += list(recommendations['user_id'])
        item_id_list += list(recommendations['item_id'])

    valid_pred = pd.DataFrame({'user_id': user_id_list, 'item_id': item_id_list})
    # display(valid_pred)
    
    ndcg_val = NDCG(validation_df, valid_pred)

    return ndcg_val

In [ ]:
# checking unique user length
len(set(validation_df['user_id']))

3466

### 1.1.3 Hyperparameter Tunning

In [ ]:
# Defining different parameters for performing hyperparameter tuning

factors = [20, 32, 64, 50]
regularization = [325, 350, 425, 475, 500, 775]
iterations = [40, 80]
alpha = list(range(50, 100, 10))

In [ ]:
import itertools

param_tuned_df = pd.DataFrame()

# for different combinations of parameters train ALS Model
for params in itertools.product(factors, regularization, iterations, alpha):
    
    model = implicit.als.AlternatingLeastSquares(factors = params[0], regularization=params[1], iterations=params[2], random_state=624)
    
    alpha = params[3] # The rate in which we'll increase our confidence
    data = (sparse_item_user * alpha).astype('double')
    
    # fitting the model
    model.fit(data)
    
    # Convert user vector and item vector obtained from model to csr_matrix
    user_vecs = sparse.csr_matrix(model.user_factors)
    item_vecs = sparse.csr_matrix(model.item_factors)
    
    print()
    print('*'*20)
    print(params)
    
    # calculate the validation NDCG score
    ndcg_value = validation_ndcg_scoring(user_vecs, item_vecs, validation_df)
    print('The NDCG score on validation dataset is: ', ndcg_value)
    
    param_tuned_df = param_tuned_df.append({'factors': params[0], 'regularization': params[1], 'iterations': params[2], 'alpha': params[3], 'ndcg_validation_score':ndcg_value}, ignore_index = True)

  0%|          | 0/40 [00:00<?, ?it/s]


********************
(20, 325, 40, 50)
The NDCG score on validation dataset is:  0.25286417966900465


  0%|          | 0/40 [00:00<?, ?it/s]


********************
(20, 325, 40, 60)
The NDCG score on validation dataset is:  0.2576258186509431


  0%|          | 0/40 [00:00<?, ?it/s]

...


********************
(50, 775, 80, 70)
The NDCG score on validation dataset is:  0.17439160491213873


  0%|          | 0/80 [00:00<?, ?it/s]


********************
(50, 775, 80, 80)
The NDCG score on validation dataset is:  0.1890257013643902


  0%|          | 0/80 [00:00<?, ?it/s]


********************
(50, 775, 80, 90)
The NDCG score on validation dataset is:  0.20438926184886153


In [ ]:
param_tuned_df.sort_values(by=['ndcg_validation_score'], ascending=False).iloc[0:20,]


,factors,regularization,iterations,alpha,ndcg_validation_score
28,20.0,425.0,80.0,80.0,0.259791
34,20.0,475.0,40.0,90.0,0.259701
39,20.0,475.0,80.0,90.0,0.259618
49,20.0,500.0,80.0,90.0,0.259566
29,20.0,425.0,80.0,90.0,0.258865
24,20.0,425.0,40.0,90.0,0.258763
6,20.0,325.0,80.0,60.0,0.258727
44,20.0,500.0,40.0,90.0,0.258719
38,20.0,475.0,80.0,80.0,0.258472
19,20.0,350.0,80.0,90.0,0.258458


### 1.1.4 Training the Final ALS Model with best Hyperparameters

In [ ]:
## Generating 10 random numbers from 0-100

import random

# to store the numbers
random_nos = []

for i in range(0,10):
    n = random.randint(1, 100)
    random_nos.append(n)

print(random_nos)

[86, 22, 99, 91, 37, 4, 61, 78, 7, 34]


In [ ]:
cc = param_tuned_df.sort_values(by=['ndcg_validation_score'], ascending=False).iloc[0:11,]

avg_ndcgs = []

for i in range(cc.shape[0]):
    
    f = int(cc.iloc[i]['factors'])
    r = int(cc.iloc[i]['regularization'])
    ii = int(cc.iloc[i]['iterations'])
    aa = int(cc.iloc[i]['alpha'])
    
    ndcg_batch_wise = []
    
    for num in random_nos:
        
        final_model = implicit.als.AlternatingLeastSquares(factors = f, regularization=r, iterations=ii, random_state=num)
    
        alpha = aa
        data = (sparse_item_user * alpha).astype('double')

        # fitting the final model
        final_model.fit(data)

        # Convert user vector and item vector obtained from final model to csr_matrix
        user_vecs = sparse.csr_matrix(final_model.user_factors)
        item_vecs = sparse.csr_matrix(final_model.item_factors)

        print('*'*20)
        # calculate the validation NDCG score
        
        ndcg_batch_wise.append(validation_ndcg_scoring(user_vecs, item_vecs, validation_df))
    
    avg_ndcgs.append(np.mean(ndcg_batch_wise))
        
avg_ndcgs

  0%|          | 0/80 [00:00<?, ?it/s]

********************


  0%|          | 0/80 [00:00<?, ?it/s]

********************


  0%|          | 0/80 [00:00<?, ?it/s]

...

  0%|          | 0/40 [00:00<?, ?it/s]

********************


  0%|          | 0/40 [00:00<?, ?it/s]

********************


[0.25821910118382724,
 0.2577007515107423,
 0.2590375372744981,
 0.2580790740903319,
 0.2582502502114338,
 0.25738888636432206,
 0.255547674662305,
 0.2569745283031935,
 0.256043455765167,
 0.2529586054093459,
 0.2571601626652432]

In [ ]:
final_model = implicit.als.AlternatingLeastSquares(factors = 20, regularization=425, iterations=80)
    
alpha = 80
data = (sparse_item_user * alpha).astype('double')

# fitting the final model
final_model.fit(data)

# Convert user vector and item vector obtained from final model to csr_matrix
user_vecs = sparse.csr_matrix(final_model.user_factors)
item_vecs = sparse.csr_matrix(final_model.item_factors)

print('*'*20)
# calculate the validation NDCG score
validation_ndcg_scoring(user_vecs, item_vecs, validation_df)

  0%|          | 0/80 [00:00<?, ?it/s]

********************


0.25746406942731614

In [ ]:
user_vecs = sparse.csr_matrix(final_model.user_factors)
item_vecs = sparse.csr_matrix(final_model.item_factors)

user_id_list = []
item_id_list = []

# for each unique user in test_df dataset
for user_id in set(test_df['user_id']):
    # recommend 15 top items
    recommendations = recommend(user_id, sparse_user_item, user_vecs, item_vecs, test_df)
    user_id_list += list(recommendations['user_id'])
    item_id_list += list(recommendations['item_id'])
    
# create output df similar to sample provided
ouput_df = pd.DataFrame({'user_id': user_id_list, 'item_id': item_id_list})

# 
ouput_df.to_csv('31072100_output.csv',index=False)

In [ ]:
test_df.shape

(346600, 2)

In [ ]:
ouput_df.shape

(51990, 2)

## 1.2 Collaborative Filtering with Neural Networks

We implement matrix factorization model and also neural network model.

In [ ]:
display(train_df.sample(5))
display(validation_df.sample(5))
display(test_df.sample(5))

,user_id,item_id,rating
75261,2353,3691,1
38803,1214,4901,1
103972,3232,3585,1
63346,1945,5070,1
61624,1889,5868,1


,user_id,item_id,rating
271754,2717,3697,0
218898,2188,1881,0
340955,3409,6867,0
73255,732,7451,0
250399,2503,8876,0


,user_id,item_id
8233,82,5590
221970,2219,2577
323241,3232,5452
149812,1498,7715
139391,1393,1005


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import implicit
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
import scipy.sparse as sparse

In [ ]:
np.random.seed(624)

### 1.2.1 Encoding Data

We enconde the data to have contiguous ids for users and itesms. You can think about this as a categorical encoding of our two categorical variables user_id and item_id.

In [ ]:
# here is a handy function modified from fast.ai
def proc_col(col, train_col=None):
    """Encodes a pandas column with continous ids. 
    """
    if train_col is not None:
        uniq = train_col.unique()
    else:
        uniq = col.unique()
    name2idx = {o:i for i,o in enumerate(uniq)}
    return name2idx, np.array([name2idx.get(x, -1) for x in col]), len(uniq)

In [ ]:
def encode_data(df, train=None):
    """ Encodes rating data with continous user and item ids. 
    If train is provided, encodes df with the same encoding as train.
    """
    df = df.copy()
    for col_name in ["user_id", "item_id"]:
        train_col = None
        if train is not None:
            train_col = train[col_name]
        _,col,_ = proc_col(df[col_name], train_col)
        df[col_name] = col
        df = df[df[col_name] >= 0]
    return df

In [ ]:
# encoding train and validation df
df_train = encode_data(train_df)
df_val = encode_data(validation_df, train_df)
display(df_train)
display(df_val)

,user_id,item_id,rating
0,0,0,1
1,0,1,1
2,0,2,1
3,0,3,1
4,0,4,1
...,...,...,...
110124,3465,8253,1
110125,3465,8809,1
110126,3465,4547,1
110127,3465,7764,1


,user_id,item_id,rating
0,0,8841,1
1,0,3115,0
2,0,378,0
3,0,5225,0
4,0,8534,0
...,...,...,...
346595,3465,8007,0
346596,3465,302,0
346597,3465,7556,0
346598,3465,5450,0


### 1.2.2 Matrix factorization model

In [ ]:
class MF(nn.Module):
    def __init__(self, num_users, num_items, emb_size=100):
        super(MF, self).__init__()
        self.user_emb = nn.Embedding(num_users, emb_size)
        self.item_emb = nn.Embedding(num_items, emb_size)
        self.user_emb.weight.data.uniform_(0, 0.05)
        self.item_emb.weight.data.uniform_(0, 0.05)
        
    def forward(self, u, v):
        u = self.user_emb(u)
        v = self.item_emb(v)
        return (u*v).sum(1)   

#### 1.2.2.1 Training MF model

In [ ]:
num_users = len(df_train.user_id.unique())
num_items = len(df_train.item_id.unique())
print(num_users, num_items)

3466 9004


In [ ]:
model = MF(num_users, num_items, emb_size=100) # .cuda() if you have a GPU

In [ ]:
def train_epocs(model, epochs=10, lr=0.01, wd=0.0, unsqueeze=False):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
    model.train()
    for i in range(epochs):
        users = torch.LongTensor(df_train.user_id.values) # .cuda()
        items = torch.LongTensor(df_train.item_id.values) #.cuda()
        ratings = torch.FloatTensor(df_train.rating.values) #.cuda()
        if unsqueeze:
            ratings = ratings.unsqueeze(1)
        y_hat = model(users, items)
        loss = F.mse_loss(y_hat, ratings)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(loss.item()) 
    test_loss(model, unsqueeze)

In [ ]:
def test_loss(model, unsqueeze=False):
    model.eval()
    users = torch.LongTensor(df_val.user_id.values) #.cuda()
    items = torch.LongTensor(df_val.item_id.values) #.cuda()
    ratings = torch.FloatTensor(df_val.rating.values) #.cuda()
    if unsqueeze:
        ratings = ratings.unsqueeze(1)
    y_hat = model(users, items)
    loss = F.mse_loss(y_hat, ratings)
    print("test loss %.3f " % loss.item())

In [ ]:
train_epocs(model, epochs=10, lr=0.1)
train_epocs(model, epochs=15, lr=0.01)
train_epocs(model, epochs=20, lr=0.01)

0.878788411617279
0.3136248290538788
0.18193288147449493
0.3102691173553467
0.08884264528751373
0.07417488098144531
0.1895395815372467
0.0019426787039265037
0.08779196441173553
0.1368265450000763
test loss 0.553 
0.05558168515563011
0.0035094004124403
0.013602501712739468
0.02641979791224003
0.015253070741891861
0.0027186591178178787
0.0013814312405884266
0.007746544666588306
0.012481150217354298
0.011217265389859676
0.0059132748283445835
0.0011400822550058365
0.00034032712574116886
0.0032277738209813833
0.006123755592852831
test loss 1.100 
0.005947473458945751
0.014421944506466389
0.006090759299695492
0.00022081284259911627
0.006234386935830116
0.006179191637784243
0.001206906046718359
0.000459829781902954
0.003303519682958722
0.004007494077086449
0.0018047402845695615
0.00014687894145026803
0.0010471649002283812
0.0024306029081344604
0.0018814810318872333
0.00042759484495036304
0.00018823910795617849
0.0011148356134071946
0.0015528240473940969
0.0008725483785383403
test loss 0.932 


#### 1.2.2.2 Prediction

In [ ]:
# encode the testing data
df_test_emb = encode_data(test_df, df_train)

# get the tesing users and items
users = torch.LongTensor(df_test_emb.user_id.values) # .cuda()
items = torch.LongTensor(df_test_emb.item_id.values)

# predict from model
Y = model(users, items)
# Scaling between 0-1
min_max = MinMaxScaler()
res_scaled = min_max.fit_transform(Y.detach().numpy().reshape(-1, 1))[:,0]
# Final prediction
res_scaled

array([0.69002056, 0.8032732 , 0.7015524 , ..., 0.77919674, 0.7360983 ,
       0.72529125], dtype=float32)

In [ ]:
# new column with predictions
test_df['pred_rating'] = list(res_scaled)

# top15 recommendation
df_test_top_15 = test_df.sort_values(by =['user_id', 'pred_rating'], ascending = (True, False))
df_test_top_15 = df_test_top_15.groupby('user_id').head(15)
df_test_top_15 = df_test_top_15.drop(['pred_rating'], axis=1)

display(df_test_top_15)

# Save Prediction to be submitted to Kaggle
df_test_top_15.to_csv( "31072100_output_mf.csv", index=False)


,user_id,item_id
46,0,4995
1,0,8906
54,0,4276
74,0,2148
39,0,5619
...,...,...
346504,3465,8763
346537,3465,5053
346580,3465,1302
346549,3465,3984


### 1.2.3 MF with bias

Bias terms describe the effect of one dimension on the output. For example, in the Netflix challenge example, the bias of a movie would describe how well this movie is rated compared to the average, across all movies. This depends only on the movie (as a first approximation) and does not take into account the interaction between an user and the movie.

Similarly an user's bias corresponds to that user's tendency to give better or worse ratings than the average.

https://stats.stackexchange.com/questions/113390/the-role-of-the-bias-terms-in-matrix-factorization-formulas/113976#:~:text=Basically%20in%20a%20factorization%20model,the%20average%2C%20across%20all%20movies.

In [ ]:
class MF_bias(nn.Module):
    def __init__(self, num_users, num_items, emb_size=100):
        super(MF_bias, self).__init__()
        self.user_emb = nn.Embedding(num_users, emb_size)
        self.user_bias = nn.Embedding(num_users, 1)
        self.item_emb = nn.Embedding(num_items, emb_size)
        self.item_bias = nn.Embedding(num_items, 1)
        self.user_emb.weight.data.uniform_(0,0.05)
        self.item_emb.weight.data.uniform_(0,0.05)
        self.user_bias.weight.data.uniform_(-0.01,0.01)
        self.item_bias.weight.data.uniform_(-0.01,0.01)
        
    def forward(self, u, v):
        U = self.user_emb(u)
        V = self.item_emb(v)
        b_u = self.user_bias(u).squeeze()
        b_v = self.item_bias(v).squeeze()
        return (U*V).sum(1) +  b_u  + b_v

In [ ]:
model = MF_bias(num_users, num_items, emb_size=100) #.cuda()

In [ ]:
train_epocs(model, epochs=10, lr=0.05, wd=1e-5)
train_epocs(model, epochs=10, lr=0.01, wd=1e-5)
train_epocs(model, epochs=10, lr=0.001, wd=1e-5)

0.8794662356376648
0.11538029462099075
0.555307149887085
0.11859992146492004
0.014549840241670609
0.1524500846862793
0.24591441452503204
0.25065547227859497
0.19293580949306488
0.1058492437005043
test loss 0.612 
0.02886783704161644
0.0035730707459151745
0.006569724529981613
0.0026941089890897274
0.0010147661669179797
0.005293488036841154
0.009051092900335789
0.00838848203420639
0.004955220501869917
0.0021885521709918976
test loss 0.912 
0.0017540794797241688
0.0014824243262410164
0.0014753134455531836
0.0015911171212792397
0.0017108379397541285
0.0017719509778544307
0.0017633449751883745
0.0017023315886035562
0.0016147885471582413
0.0015238961204886436
test loss 0.879 


Note that these models are susceptible to weight initialization, optimization algorithm and regularization.

#### 1.2.3.1 Prediction

In [ ]:
# encode the testing data
df_test_emb = encode_data(test_df, df_train)

# get the tesing users and items
users = torch.LongTensor(df_test_emb.user_id.values) # .cuda()
items = torch.LongTensor(df_test_emb.item_id.values)

# predict from model
Y = model(users, items)
# Scaling between 0-1
min_max = MinMaxScaler()
res_scaled = min_max.fit_transform(Y.detach().numpy().reshape(-1, 1))[:,0]
# Final prediction
res_scaled

array([0.599349  , 0.5200224 , 0.55882215, ..., 0.5238857 , 0.6207881 ,
       0.5949545 ], dtype=float32)

In [ ]:
# new column with predictions
test_df['pred_rating'] = list(res_scaled)

# top15 recommendation
df_test_top_15 = test_df.sort_values(by =['user_id', 'pred_rating'], ascending = (True, False))
df_test_top_15 = df_test_top_15.groupby('user_id').head(15)
df_test_top_15 = df_test_top_15.drop(['pred_rating'], axis=1)

display(df_test_top_15)

# Save Prediction to be submitted to Kaggle
df_test_top_15.to_csv( "31072100_output_mfbias.csv", index=False)


,user_id,item_id
91,0,838
97,0,349
64,0,3177
46,0,4995
30,0,6363
...,...,...
346531,3465,6120
346565,3465,2722
346557,3465,3581
346551,3465,3889


## 1.3 References

* Lecture Slides

* Tutorial Codes

* https://medium.com/radon-dev/als-implicit-collaborative-filtering-5ed653ba39fe

* https://ieeexplore.ieee.org/document/4781121

* https://stats.stackexchange.com/questions/155296/alternating-least-squares-als-why-two-different-kinds-of-setting-for-lambd

* https://stats.stackexchange.com/questions/113390/the-role-of-the-bias-terms-in-matrix-factorization-formulas/113976#:~:text=Basically%20in%20a%20factorization%20model,the%20average%2C%20across%20all%20movies

* https://towardsdatascience.com/prototyping-a-recommender-system-step-by-step-part-2-alternating-least-square-als-matrix-4a76c58714a1#:~:text=Alternating%20Least%20Square%20(ALS)%20with%20Spark%20ML&text=Some%20high%2Dlevel%20ideas%20behind%20ALS%20are%3A&text=Its%20training%20routine%20is%20different,gradient%20descent%20with%20user%20matrix


---

--- 

# Task 2: Node Clustering in Graphs

---

**Student Name:** Abhi Gambhir

**Student ID:** 31072100

In [ ]:
# installing torch
!pip install torch

## 2.1 Importing Libraries

In [ ]:
# importing libraries with seeding
import numpy as np
np.random.seed(123)

import torch
torch.manual_seed(123)

In [ ]:
# importing some more libraries libraries
import time

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import scipy as sp
import scipy.sparse.linalg as linalg
import scipy.cluster.hierarchy as hr
from scipy.spatial.distance import pdist, squareform

import sklearn.datasets as datasets
import sklearn.metrics as metrics
import sklearn.utils as utils
import sklearn.linear_model as linear_model
import sklearn.svm as svm
import sklearn.cluster as cluster
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler

# importing networkx library
import networkx as nx

import seaborn as sns
%matplotlib inline

## 2.2 Reading .txt files

In [ ]:
# reading docs.txt and storing it in a dataframe

docs = open('Task2_GraphData/docs.txt')
docs_df = pd.DataFrame()

# for each line in txt file
for i in docs.readlines():
    
    # split the line to get node_id and title
    # setting the maxsplit parameter to 1, will return a list with 2 elements 
    # append to empty dataframe
    docs_df = docs_df.append({'node_id': i.split(' ',1)[0].strip(), 'title': i.split(' ',1)[1].strip()}, ignore_index = True)

# displaying the made dataframe
docs_df

,node_id,title
0,12828558,"Assessing Local Institutional Capacity, Data A..."
1,66779408,THE PROSPECTS FOR INTERNET TELEPHONY IN EUROPE...
2,38902949,"Economic Shocks, Safety Nets, and Fiscal Const..."
3,33450563,"Reform, Growth, and Poverty in Vietnam"
4,57470294,Households and Economic Growth in Latin Americ...
...,...,...
18715,39790331,ORIGINAL ARTICLE Effect of Alzheimer Disease R...
18716,47870669,BRIEF REPORTS Predictors of Posttraumatic Stre...
18717,28485685,Anticipating the cultural aspects of sharing f...
18718,48611755,BMC Psychiatry BioMed Central


In [ ]:
# reading labels.txt and storing it in a dataframe

labels = open('Task2_GraphData/labels.txt')

labels_df = pd.DataFrame()

# for each line in the txt file
for i in labels.readlines():
    
    # strip (to remove /n) and split on ' ' (empty space)
    text_split = i.strip().split(' ')
    
    # append to empty dataframe
    labels_df = labels_df.append({'node_id': int(text_split[0]), 'class_label': int(text_split[1])}, ignore_index = True)

# displaying the made dataframe
labels_df

,node_id,class_label
0,12828558.0,0.0
1,66779408.0,0.0
2,38902949.0,0.0
3,33450563.0,0.0
4,57470294.0,0.0
...,...,...
18715,39790331.0,4.0
18716,47870669.0,4.0
18717,28485685.0,4.0
18718,48611755.0,4.0


## 2.3 Datatype conversion of some columns

In [ ]:
labels_df = labels_df.astype(int)
display(labels_df.head())

docs_df['node_id'] = docs_df['node_id'].astype(int)
display(docs_df.head())

,node_id,class_label
0,12828558,0
1,66779408,0
2,38902949,0
3,33450563,0
4,57470294,0


,node_id,title
0,12828558,"Assessing Local Institutional Capacity, Data A..."
1,66779408,THE PROSPECTS FOR INTERNET TELEPHONY IN EUROPE...
2,38902949,"Economic Shocks, Safety Nets, and Fiscal Const..."
3,33450563,"Reform, Growth, and Poverty in Vietnam"
4,57470294,Households and Economic Growth in Latin Americ...


## 2.4 Merging labels and docs dataframe

In [ ]:
merged_df = labels_df.merge(docs_df, on='node_id')
print(merged_df.shape)
merged_df.head()

(18720, 3)


,node_id,class_label,title
0,12828558,0,"Assessing Local Institutional Capacity, Data A..."
1,66779408,0,THE PROSPECTS FOR INTERNET TELEPHONY IN EUROPE...
2,38902949,0,"Economic Shocks, Safety Nets, and Fiscal Const..."
3,33450563,0,"Reform, Growth, and Poverty in Vietnam"
4,57470294,0,Households and Economic Growth in Latin Americ...


In [ ]:
# Checking unique values for true class labels
merged_df['class_label'].unique()

array([0, 1, 2, 3, 4])

> So unique True Class labels are 0,1,2,3,4 i.e. our K should become 5. We will be using K=5 for our clustering algorithmns later

## 2.5 Creating Graph using Adjedges

In [ ]:
# Reads graph in adjacency list format from path 
G = nx.read_adjlist('Task2_GraphData/adjedges.txt', nodetype = int)


In [ ]:
print(len(G.nodes()))
print(len(G.edges()))

36928
54183


> There are 36928 nodes present and 54183 edges in the network. But if you notice we only have true class labels for 18720 nodes only. We have to keep this in mind while scoring i.e. NMI scoring our algorithm

In [ ]:
# No nulls are there (just random check)
merged_df[merged_df['class_label'].isna()]


,node_id,class_label,title


In [ ]:
nodes_with_labels = set(list(merged_df['node_id']))
print(len(nodes_with_labels))

graph_nodes = set(list(G.nodes()))
print(len(graph_nodes))

18720
36928


> Double confirms the point raised above.

In [ ]:
count = 0

for subgraph in nx.connected_components(G):
    count += 1
    
count

10440

> Number of subgraphs present in graph is 10440

## 2.6 Generating 10 random numbers

In [ ]:
## Generating 10 random numbers from 0-100

import random

# to store the numbers
random_nos = []

for i in range(0,10):
    n = random.randint(1, 100)
    random_nos.append(n)

print(random_nos)

[14, 45, 25, 18, 86, 23, 8, 91, 33, 82]


---

## 2.7 Algo 1: Spectral Clustering

### 2.7.1 Step1: Preprocessing and get the laplacian matrix.

In [ ]:
L = nx.laplacian_matrix(G).astype(float)

### 2.7.2 Step2: Decomposition - Compute eigenvalues and eigenvectors of the matrix.

In [ ]:
w,v = sp.sparse.linalg.eigsh(L, k = 5, which='SM')

In [ ]:
print(w)
print(v)
print(w.shape, v.shape)

[-4.06825556e-14 -4.02127600e-14 -2.20296202e-14  1.22096105e-14
  3.41550107e-14]
[[-1.99632999e-03  1.08479446e-04  2.39063918e-03 -6.68050338e-03
  -4.90417305e-04]
 [ 8.75117874e-06 -2.81973588e-07 -9.33580741e-06 -1.46200324e-05
  -2.36486001e-06]
 [-1.33503075e-02 -7.17775128e-03  2.35793121e-02  6.19076659e-03
  -5.34248751e-03]
 ...
 [ 8.19047960e-04  1.00212525e-02  5.07683534e-03 -6.41974280e-03
  -7.19694814e-03]
 [ 4.38075005e-03  1.09437075e-03 -1.26258321e-03  9.75217111e-03
  -5.22804062e-03]
 [-1.43266207e-03 -1.38307159e-03 -1.30427345e-03  1.03396344e-02
  -3.58348050e-03]]
(5,) (36928, 5)


In [ ]:
X = v*w

### 2.7.3 Step3: K Means Clustering

In [ ]:
# importing normalized_mutual_info_score for NMI scoring and KMeans algo from sklearn
from sklearn.metrics.cluster import normalized_mutual_info_score
from sklearn.cluster import KMeans

# empty list to store nmi_scores for each random_state
nmi_scores = []

# for each random number perform KMeans clustering
for number in random_nos:
    
    # n_clusters = 5 as we had 5 unique true labels of nodes
    kmeans = KMeans(init='k-means++', n_clusters=5, n_init=50, random_state=number)
    kmeans.fit_predict(X)
    km_centroids = kmeans.cluster_centers_
    km_labels = kmeans.labels_    # predicted labels
    km_error = kmeans.inertia_

    # creating dataframe of all trained nodes and its predicted label
    pred_df = pd.DataFrame({'node_id': [node_id for node_id in G.nodes()], 'pred_label': list(km_labels)})

    # Since we had less number of nodes with true labels, we perform inner join of our merged_df
    # with the one above generated

    final_df = pd.merge(merged_df, pred_df, how='inner', on='node_id')
    
    # generating nmi score of the nodes we have and appending in to the nmi_scores list
    nmi = normalized_mutual_info_score(final_df['class_label'].values, final_df['pred_label'].values)
    nmi_scores.append(nmi)

# printing nmi scores obtained
nmi_scores

[0.13368250220554137,
 0.1337528144064061,
 0.13372126481121657,
 0.13371598272407473,
 0.1339362910770202,
 0.1336307013039777,
 0.13375063126236125,
 0.13372126481121654,
 0.13387387146521865,
 0.13360687239049984]

In [ ]:
# avg of nmi_scores
print('Average/Mean NMI of 10 random_state trained K-Means model: ', np.mean(nmi_scores))

Average/Mean NMI of 10 random_state trained K-Means model:  0.13373921964575328


---

## 2.8 Algo 2: Node2Vec

In [ ]:
# installation of nord2vec
!pip install Node2Vec

Some Node2Vec Parameters:

* :param dimensions: Embedding dimensions
* :param walk_length: Number of nodes in each walk
* :param num_walks: Number of walks per node

In [ ]:
from node2vec import Node2Vec

# Import pickle Package, to store the model (as it takes time to train)
import pickle

#pre-compute the probabilities and generate walks :
node2vec = Node2Vec(G, dimensions=32, walk_length=10, num_walks=20)

#embed the nodes
model = node2vec.fit(window=10, min_count=1, batch_words=4)

Computing transition probabilities:   0%|          | 0/36928 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|████████████████| 20/20 [00:30<00:00,  1.52s/it]


In [ ]:
# saving the loading the model
with open("node2vec_model.pkl", 'wb') as file:  
    pickle.dump(model, file)

# reading the saved model
with open("node2vec_model.pkl", 'rb') as file:  
    model_n2v = pickle.load(file)

In [ ]:
# checking
model_n2v

In [ ]:
# to store the embeddings of each node
embeddings = []

# empty list to store nmi_scores for each random_state
nmi_scores = []

for node in G.nodes():
    embeddings.append(model_n2v.wv.get_vector(str(node)))

for number in random_nos:
    
    # n_clusters = 5 as we had 5 unique true labels of nodes
    kmeans = KMeans(init='k-means++', n_clusters=5, n_init=50, random_state=number)
    kmeans.fit_predict(embeddings)
    km_centroids = kmeans.cluster_centers_
    km_labels = kmeans.labels_     # predicted labels
    km_error = kmeans.inertia_

    # creating dataframe of all trained nodes and its predicted label
    pred_df = pd.DataFrame({'node_id': [node_id for node_id in G.nodes()], 'pred_label': list(km_labels)})
    
    # Since we had less number of nodes with true labels, we perform inner join of our merged_df
    # with the one above generated
    
    final_df = pd.merge(merged_df, pred_df, how='inner', on='node_id')

    # generating nmi score of the nodes we have and appending in to the nmi_scores list
    nmi = normalized_mutual_info_score(final_df['class_label'].values, final_df['pred_label'].values)
    nmi_scores.append(nmi)
    
# printing nmi scores obtained
nmi_scores

[0.3453062608226242,
 0.34531825652040743,
 0.3451980383435631,
 0.3452055157476496,
 0.34502535910443954,
 0.3452003374129036,
 0.3449931763246772,
 0.3448969337362679,
 0.3445640629374805,
 0.34525953945024873]

In [ ]:
# avg of nmi_scores
print('Average/Mean NMI of 10 random_state trained K-Means model: ', np.mean(nmi_scores))

Average/Mean NMI of 10 random_state trained K-Means model:  0.3450967480400262


## 2.9 References

* Lecture Slides

* Tutorial Codes

* https://snap.stanford.edu/class/cs224w-2017/projects/cs224w-38-final.pdf

* https://towardsdatascience.com/complete-guide-to-understanding-node2vec-algorithm-4e9a35e5d147

* https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.eigsh.html

* https://scikit-learn.org/stable/modules/generated/sklearn.metrics.normalized_mutual_info_score.html#:~:text=Normalized%20Mutual%20Information%20(NMI)%20is,and%201%20(perfect%20correlation)

* https://arxiv.org/pdf/1809.00979.pdf